# Your first convolutional neural network
# imports


In [21]:
import torch
from torch import nn
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from tqdm.notebook import tqdm

# data

In [22]:
NUM_CLASSES = 10

In [23]:
train_data = CIFAR10('~/pytorch', train=True, transform=ToTensor(), download=True)
train_loader = DataLoader(train_data, batch_size=32)
test_data = CIFAR10('~/pytorch', train=False, transform=ToTensor(), download=True)
test_loader = DataLoader(test_data, batch_size=32)

Files already downloaded and verified
Files already downloaded and verified


# architecture

In [28]:
model = nn.Sequential(
    nn.Conv2d(3, 10, 4, stride=2, padding=1),
    nn.Conv2d(10, 20, 3, stride=2, padding=1),
    nn.Flatten(),
    nn.Linear(1280, 10),
    nn.Softmax())
print(model)

Sequential(
  (0): Conv2d(3, 10, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): Conv2d(10, 20, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Flatten()
  (3): Linear(in_features=1280, out_features=10, bias=True)
  (4): Softmax(dim=None)
)


# train

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = torch.nn.BCELoss(reduction='sum')

In [30]:
for t in range(10):
    print("Epoch {}/10".format(t + 1))
    total_loss = 0
    for x, y in tqdm(train_loader):
        output = model(x)
        y_pred = one_hot(y, NUM_CLASSES).float()
        loss = criterion(output, y_pred)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Loss: {:4f}".format(total_loss / len(train_data)))

Epoch 1/10



Loss: 2.725999
Epoch 2/10



Loss: 2.583565
Epoch 3/10



Loss: 2.559222
Epoch 4/10



Loss: 2.545675
Epoch 5/10


...


Loss: 2.528651
Epoch 7/10



Loss: 2.522078
Epoch 8/10



Loss: 2.516101
Epoch 9/10



Loss: 2.510550
Epoch 10/10



Loss: 2.505372


# analysis

In [27]:
model.eval()
correct = 0
x = 0.0
y = 1.0
for test_x, test_y in tqdm(test_loader):
    output = model(test_x)
    y_pred = output.argmax(dim=1)
    correct += (y_pred == test_y).float().sum()
print("Accuracy: {:.4f}".format(correct / len(test_data)))


Accuracy: 0.4168
